# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [21]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [29]:
# 1) Seu código aqui

URL = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'
r = requests.get(URL)
r.status_code

200

In [30]:
data_json = r.json()
data_json.keys()

dict_keys(['items', 'hasMore', 'limit', 'offset', 'count', 'links'])

In [31]:
df = pd.DataFrame(data_json['items'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_pleito              221 non-null    int64  
 1   tipo_interessado       221 non-null    object 
 2   interessado            221 non-null    object 
 3   cod_ibge               221 non-null    int64  
 4   uf                     221 non-null    object 
 5   num_pvl                44 non-null     object 
 6   status                 221 non-null    object 
 7   num_processo           206 non-null    object 
 8   data_protocolo         206 non-null    object 
 9   tipo_operacao          221 non-null    object 
 10  finalidade             221 non-null    object 
 11  tipo_credor            221 non-null    object 
 12  credor                 221 non-null    object 
 13  moeda                  221 non-null    object 
 14  valor                  221 non-null    float64
 15  pvl_as

In [7]:
# 2) Seu código aqui
df_3status = df['status'].value_counts()[:3].to_frame()
df_3status

,status
Deferido,86
Arquivado,53
Regularizado,25


In [8]:
# 3) Seu código aqui
df['data_status_ano'] = pd.to_datetime(df['data_status']).dt.year
df.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status,data_status_ano
0,10745,Município,Angra dos Reis,3300100,RJ,None,Deferido,17944.001273/2008-80,2009-07-24T03:00:00Z,Operação contratual interna,PAC - Saneamento - 2008 B,Instituição Financeira Nacional,Caixa Econômica Federal,Real,20177032.0,0,0,29/07/2009,2009
1,10226,Município,Conceição de Macabu,3301405,RJ,None,Arquivado,17944.001149/2008-14,2008-08-14T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,400700.0,0,0,30/10/2008,2008


In [9]:
# 4) Seu código aqui
df_anostatus = df['data_status_ano'].value_counts().to_frame()
df_anostatus

,data_status_ano
2008,37
2007,29
2014,20
2010,19
2019,17
2020,15
2013,15
2012,13
2011,10
2015,9


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [34]:
#1) Seu código aqui
# url1 = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado'
# url2 = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

def buscar_pvl(uf:str,interessado:str) -> pd.DataFrame:
    interessado = 'Município' if interessado == 'Municipio' else interessado
    url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=' + uf + '&tipo_interessado=' + interessado
    r = requests.get(url)
    return pd.DataFrame(r.json()['items'])

In [44]:
# 2) Seu código aqui

df = buscar_pvl('MG','Estado')
df[df['status'] == 'Arquivado por decurso de prazo'].shape[0]

1

In [57]:
# 3) Seu código aqui
df = buscar_pvl('BA','Municipio')
df[df['status'] == 'Deferido']['interessado'].value_counts().to_frame().iloc[0]

interessado    14
Name: Luís Eduardo Magalhães, dtype: int64

In [61]:
# 4) Seu código aqui
buscar_pvl('BA','Estado').to_csv('PVL Bahia.csv',sep=';',index='False',encoding='UTF8')